# Assignment 4: Pipelines and Hyperparameter Tuning (32 total marks)
### Due: November 22 at 11:59pm

### Name: Billy Sidharta

### In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models and evaluate the results. More details for each step can be found below.

### You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.

## Import Libraries

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

## Step 1: Data Input (4 marks)

Import the dataset you will be using. You can download the dataset onto your computer and read it in using pandas, or download it directly from the website. Answer the questions below about the dataset you selected. 

To find a dataset, you can use the resources listed in the notes. The dataset can be numerical, categorical, text-based or mixed. If you want help finding a particular dataset related to your interests, please email the instructor.

**You cannot use a dataset that was used for a previous assignment or in class**

In [10]:
# Import dataset (1 mark)
df = pd.read_csv('reduced_dataset.csv')
df.shape

(66325, 221)

In [11]:
print("The total number of null values in the dataset: ", sum(df.isna().sum()))
pd.set_option('display.max_columns', None)
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

The total number of null values in the dataset:  0


,AllowCharge,AllowDischarge,AuxBmsAlive,AuxVoltage,ChargeContactorError,ChargeNotClosedDueToHighCurrent,ChargeOpenButShouldBeClosed,ChargeShouldTrip,ChargeTripDueToHighCellVoltage,ChargeTripDueToHighTemperatureAndCurrent,ChargeTripDueToPackCurrent,CommonContactorError,DischargeContactorError,DischargeNotClosedDueToHighCurrent,DischargeOpenButShouldBeClosed,DischargeShouldTrip,DischargeTripDueToHighTemperatureAndCurrent,DischargeTripDueToLowCellVoltage,DischargeTripDueToPackCurrent,HighVoltageEnableState,OrionCANReceivedRecently,PrechargeState,ProtectionTrip,StrobeBmsLight,TripDueToContactorDisconnectedUnexpectedly,TripDueToOrionMessageTimeout,12vInputVoltage,Alive_x,AverageCellVoltage,AverageTemperature,FanSpeed,FanVoltage,HighCellVoltage,HighCellVoltageId,HighTemperature,HighThermistorId,InternalTemperature,LowCellVoltage,LowCellVoltageId,LowTemperature,LowThermistorId,PackAmphours,PackCurrent,PackDepthOfDischarge,PackStateOfCharge,PackVoltage,PopulatedCells,RequestedFanSpeed,BMSRelayStatusFlags.AlwaysOnSignalStatus,BMSRelayStatusFlags.ChargeRelayEnabled,BMSRelayStatusFlags.ChargerSafetyEnabled,BMSRelayStatusFlags.DischargeRelayEnabled,BMSRelayStatusFlags.IsChargingSignalStatus,BMSRelayStatusFlags.IsReadySignalStatus,BMSRelayStatusFlags.MalfunctionIndicatorActive,BMSRelayStatusFlags.MultiPurposeInputSignalStatus,ErrorFlags.12vPowerSupplyFault,ErrorFlags.AlwaysOnSupplyFault,ErrorFlags.CANBUSCommunicationsFault,ErrorFlags.ChargeLimitEnforcementFault,ErrorFlags.ChargerSafetyRelayFault,ErrorFlags.CurrentSensorFault,ErrorFlags.DischargeLimitEnforcementFault,ErrorFlags.FanMonitorFault,ErrorFlags.HighVoltageIsolationFault,ErrorFlags.InternalCommunicationFault,ErrorFlags.InternalConversionFault,ErrorFlags.InternalLogicFault,ErrorFlags.InternalMemoryFault,ErrorFlags.InternalThermistorFault,ErrorFlags.LowCellVoltageFault,ErrorFlags.OpenWiringFault,ErrorFlags.PackVoltageSensorFault,ErrorFlags.ThermistorFault,ErrorFlags.VoltageRedundancyFault,ErrorFlags.WeakCellFault,ErrorFlags.WeakPackFault,LimitFlags.CclReducedDueToAlternateCurrentLimit,LimitFlags.CclReducedDueToChargerLatch,LimitFlags.CclReducedDueToHighCellResistance,LimitFlags.CclReducedDueToHighCellVoltage,LimitFlags.CclReducedDueToHighPackVoltage,LimitFlags.CclReducedDueToHighSoc,LimitFlags.CclReducedDueToTemperature,LimitFlags.DclReducedDueToHighCellResistance,LimitFlags.DclReducedDueToLowCellVoltage,LimitFlags.DclReducedDueToLowPackVoltage,LimitFlags.DclReducedDueToLowSoc,LimitFlags.DclReducedDueToTemperature,LimitFlags.DclandCclReducedDueToCommunicationFailsafe,LimitFlags.DclandCclReducedDueToVoltageFailsafe,CcsAlive,Acceleration,Alive_y,Aux,Brakes_x,Forward,Hazard,HeadlightsHigh,HeadlightsLow,HeadlightsOff,Horn,Interior,Lap,NextSong,PrevSong,PushToTalk,RegenBraking,Reset,Reverse,SignalLeft,SignalRight,VolumeDown,VolumeUp,L.Alive,L.BusCurrent,L.BusVoltage,L.SetCurrent,L.SetVelocity,L.VehicleVelocity,R.Alive,R.BusCurrent,R.BusVoltage,R.SetCurrent,R.SetVelocity,R.VehicleVelocity,Alive,BmsStrobeLight,Brakes_y,HighBeams,LeftSignal,LowBeams,RightSignal,MPPT1.Alive,MPPT1.ArrayCurrent,MPPT1.ArrayVoltage,MPPT1.BatteryVoltage,MPPT1.Temperature,MPPT2.Alive,MPPT2.ArrayCurrent,MPPT2.ArrayVoltage,MPPT2.BatteryVoltage,MPPT2.Temperature,MPPT3.Alive,MPPT3.ArrayCurrent,MPPT3.ArrayVoltage,MPPT3.BatteryVoltage,MPPT3.Temperature,MPPT4.Alive,MPPT4.ArrayCurrent,MPPT4.ArrayVoltage,MPPT4.BatteryVoltage,MPPT4.Temperature,L.BackEmf,L.DcBusAmpHours,L.DspBoardTemp,L.HeatSinkTemp,L.MotorCurrentImaginary,L.MotorCurrentReal,L.MotorTemp,L.MotorVoltageImaginary,L.MotorVoltageReal,L.Odometer,L.PhaseBCurrent,L.PhaseCCurrent,L.SlipSpeed,L.VoltageRail15VSupply,L.VoltageRail1VSupply,L.VoltageRail3VSupply,R.BackEmf,R.DcBusAmpHours,R.DspBoardTemp,R.HeatSinkTemp,R.MotorCurrentImaginary,R.MotorCurrentReal,R.MotorTemp,R.MotorVoltageImaginary,R.MotorVoltageReal,R.Odometer,R.PhaseBCurrent,R.PhaseCCurrent,R.SlipSpeed,R.VoltageRail15VSupply,R.VoltageRail1VSupply,R.VoltageRail3VSupply,L.RxErrorCount,L.TxErrorCount,L.ErrorFla

### Questions (3 marks)

1. (1 mark) What is the source of your dataset?
1. (1 mark) Why did you pick this particular dataset?
1. (1 mark) Was there anything challenging about finding a dataset that you wanted to use?

*ANSWER HERE*

1. The source of my data is from the Solar Car Team, collected by the telemetry team.
2. I picked this dataset because I find it interesting and relevant to the work I do for the team (battery pack team). I hope to use what I have started hear to expand and find more useful uses from the data collected during race.
3. Not challenging to find once I thought of it. 

## Step 2: Data Processing (5 marks)

The next step is to process your data. Implement the following steps as needed.

In [12]:
# Clean data (if needed)

# Derive target variable which is 'BatteryPackCapacity' from 'PackAmpHours' and 'PackVoltage' 
# Feature matrix will be all columns except for 'BatteryPackCapacity', 'PackAmpHours' and 'PackVoltage' 

df['BatteryPackCapacity'] = df['PackAmphours'] * df['PackVoltage'] / 1000 
non_feature_columns = ['BatteryPackCapacity', 'PackAmphours', 'PackVoltage']
df.shape
df = df[df['TimeStamp'] > 1.6570025E12]
X = df.drop(columns=non_feature_columns).copy()
y = df.BatteryPackCapacity

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [13]:
#Column Transformer
feature_columns = X.select_dtypes(include=['bool']).columns
numeric_columns = X.select_dtypes(include=['float', 'int']).columns
column_trans = make_column_transformer(
    (StandardScaler(), numeric_columns.values),
    ('passthrough', feature_columns),
    (OneHotEncoder(), ['PrechargeState']),
    remainder='drop'
    )

### Questions (2 marks)

1. (1 mark) Were there any missing/null values in your dataset? If yes, how did you replace them and why? If no, describe how you would've replaced them and why.
2. (1 mark) What type of data do you have? What preprocessing methods would you have to apply based on your data types?

*ANSWER HERE*
1. I did not find any null values in the dataset (which was actually surprising), if I did find null values, the approach would highly depend on the feature. For example, if it was pack temperature, it may be appropriate to use the previous value since temperature does not vary much, especially since each record is taken every 0.5ms. 
2. There were 4 types of data:
    - float: I used a standard scaler since I knew I wanted to implemented Lasso and the value range widely between features. 
    - int: I used a standard scaler since I knew I wanted to implemented Lasso and the value range widely between features.  
    - bool: No preprocessing required.
    - object: I used a one hot encoder to generate 4 numeric features that a model can process.

## Step 3: Implement Machine Learning Model (11 marks)

In this section, you will implement three different supervised learning models (one linear and two non-linear) of your choice. You will use a pipeline to help you decide which model and hyperparameters work best. It is up to you to select what models to use and what hyperparameters to test. You can use the class examples for guidance. You must print out the best model parameters and results after the grid search.

In [14]:
# Implement pipeline and grid search here. Can add more code blocks if necessary

### Lasso

In [15]:
lasso = Lasso(max_iter=2000)
pipe_lasso = make_pipeline(column_trans, lasso) # takes data, transform column using specified preprocessing in column transformer and build the model
parameters_lasso = {}
parameters_lasso['lasso__alpha'] = [0.01, 0.1, 1, 10, 100]
grid_lasso = GridSearchCV(pipe_lasso, parameters_lasso, cv=5, scoring= 'neg_mean_squared_error' )
grid_lasso.fit(X_train, y_train)
best_params_values_lasso = list(grid_lasso.best_params_.values())
print(best_params_values_lasso)
grid_lasso.best_params_

[0.01]


{'lasso__alpha': 0.01}

In [16]:
train_rms_lasso = -grid_lasso.best_score_
test_rms_lasso = -grid_lasso.score(X_test, y_test)
lasso = Lasso(alpha=best_params_values_lasso[0] ,max_iter=2000)
train_r2_score_lasso = cross_val_score(pipe_lasso, X_train, y_train, cv=5, scoring='r2').mean()

### Decision Tree

In [17]:
np.random.seed(42)
tree_regressor = DecisionTreeRegressor()
pipe_tree = make_pipeline(column_trans, tree_regressor)
parameters_tree = {
    'decisiontreeregressor__max_depth': [5, 20, 50],
    'decisiontreeregressor__min_samples_split': [2, 5, 10],
    'decisiontreeregressor__min_samples_leaf' : [1, 5, 10]
}
grid_tree = GridSearchCV(pipe_tree, parameters_tree, cv=2, scoring='neg_mean_squared_error')
grid_tree.fit(X_train, y_train)
best_params_values_tree = list(grid_tree.best_params_.values())
print(best_params_values_tree)
grid_tree.best_params_

[20, 1, 5]


{'decisiontreeregressor__max_depth': 20,
 'decisiontreeregressor__min_samples_leaf': 1,
 'decisiontreeregressor__min_samples_split': 5}

In [18]:
train_rms_tree = -grid_tree.best_score_
test_rms_tree = -grid_tree.score(X_test, y_test)
tree_regressor = DecisionTreeRegressor(max_depth=best_params_values_tree[0], min_samples_leaf=best_params_values_tree[1], min_samples_split=best_params_values_tree[2])
train_r2_score_tree = cross_val_score(pipe_tree, X_train, y_train, cv=5, scoring='r2').mean()

### Random Forrest Regressor


In [19]:
np.random.seed(42)
gradient_regressor = GradientBoostingRegressor()
pipe_gradient = make_pipeline(column_trans, gradient_regressor)
parameters_gradient = { 
    # Some parameters were not taken into account to decrease computation time. 
    #'gradientboostingregressor__n_estimators': [50, 100, 200],
    'gradientboostingregressor__n_estimators': [10, 20],
    'gradientboostingregressor__learning_rate': [0.01, 0.1],
    'gradientboostingregressor__max_depth': [3, 5],
    #'gradientboostingregressor__min_samples_split': [2, 5, 10],
    #'gradientboostingregressor__min_samples_leaf': [1, 5, 10]
}
grid_gradient = GridSearchCV(pipe_gradient, parameters_gradient, cv=2, scoring='neg_mean_squared_error')
grid_gradient.fit(X_train, y_train)
best_params_values_gradient = list(grid_gradient.best_params_.values())
print(best_params_values_gradient)
grid_gradient.best_params_

[0.1, 5, 20]


{'gradientboostingregressor__learning_rate': 0.1,
 'gradientboostingregressor__max_depth': 5,
 'gradientboostingregressor__n_estimators': 20}

In [20]:
train_rms_gradient = -grid_gradient.best_score_
test_rms_gradient = -grid_gradient.score(X_test, y_test)
gradient_gradient = GradientBoostingRegressor(learning_rate=best_params_values_gradient[0], max_depth=best_params_values_gradient[1], n_estimators=best_params_values_gradient[2])
train_r2_score_gradient = cross_val_score(pipe_gradient, X_train, y_train, cv=2, scoring='r2').mean()

### Questions (5 marks)

1. (1 mark) Do you need regression or classification models for your dataset?
2. (2 marks) Which models did you select for testing and why?
3. (2 marks) Which model worked the best? Does this make sense based on the theory discussed in the course and the context of your dataset?

*ANSWER HERE*
1. I needed regression since the target was battery pack capacity which is a numerical value in kWh which ranges between 0-18 kWh.
2. I selected:
    - Lasso: Selected because I knew I had a lot of features that may be irrelevant and was interested in the normalization that lasso does which assigns some features a coef of zero to reduce complexity. 
    - Decision Tree: I was interested to see how it would perform since it is a widely used model.
    - Gradient Boosted: Known to be very powerful and works well with scaled features and binary features, interested in how it performs.
3. Lasso worked the best and it makes sense because I used a lot of features and I know many of them have no correlation to the target. Lasso's ability to give some features a weight of zero is perfect to reduce the model's complexity to avoid overfitting.

## Step 4: Validate Model (6 marks)

Use the testing set to calculate the testing accuracy for the best model determined in Step 3.

In [21]:
# Calculate testing accuracy (1 mark)
results = pd.DataFrame(columns=['Model', 'Best Parameter', 'CrossVal R2', 'Train RMS', 'Test RMS'])
results.loc[0] = ['Lasso', grid_lasso.best_params_, train_r2_score_lasso, train_rms_lasso, test_rms_lasso]
results.loc[1] = ['Decision Tree', grid_tree.best_params_, train_r2_score_tree, train_rms_tree, test_rms_tree]
results.loc[2] = ['Gradient Boosted Tree', grid_gradient.best_params_, train_r2_score_gradient, train_rms_gradient, test_rms_gradient]
results

,Model,Best Parameter,CrossVal R2,Train RMS,Test RMS
0,Lasso,{'lasso__alpha': 0.01},0.960523,0.011341,0.011126
1,Decision Tree,"{'decisiontreeregressor__max_depth': 20, 'deci...",0.999995,0.000211,0.000110
2,Gradient Boosted Tree,{'gradientboostingregressor__learning_rate': 0...,0.999956,0.400540,0.400093



### Questions (5 marks)

1. (1 mark) Which accuracy metric did you choose? 
1. (1 mark) How do these results compare to those in part 3? Did this model generalize well?
1. (3 marks) Based on your results and the context of your dataset, did the best model perform "well enough" to be used out in the real-world? Why or why not? Do you have any suggestions for how you could improve this analysis?

*ANSWER HERE*
1. The main accuracy metric used was RMSE because I care about the difference between the predicted value and the actual value of the battery pack capacity. Future data collected should be between the provided range so as long as it can properly predict the validation set (called testing in this assignment), it indicates good performance. I also used R^2 score to gain insight on the model's fit (overfitting or underfitting).
2. The models received a R2 score very close to one which means they are overfitting. However it also leads to very low RMS scores which leads me to question if the model is overfitted or the large number of samples trained the model very well. I may need to further explore this since its my first time working with somewhat large dataset.
3. Based on the model and my current knowledge, I would select lasso regression since the R2 score at 0.96 is high but not in the 0.99s which means the model is not too overfitted while still producing very good training and testing rms scores. The model performed very well in my opinion and I would definitely considering using it in the real world if applicable. For this dataset in particular, an rms score of 0.01 is definitely acceptable for a battery pack. The performance can be further improved by exploring more parameter possibilities for decision tree regression, this was reduced due to computation time. Also increasing cross val could provide more insight/reliability but was also reduced due to computation power.

## Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE*
1. I sourced my code from the UofC Solar Car Team, data collected by the telemetry team. 
2. I completed the steps in order for a single model (Lasso), once that was figured out, I repeated the pipeline and gridsearch for tree and random forest. I did have to go back to the data and reduced the number of samples so it had a reasonable runtime. 
3. I did use generative AI to assist me with dealing with the large dataset, it was my first time working with a large dataset (3.5Gb json file) which had a nested format, which I had to reduce to use for this assignment. I dont recall the exact prompt but it was along the lines of "given a large json file, convert to panda dataframe", then "normalize nested format for a panda dataframe". I did have to modify a lot of the code for my specific format such as number of nested arguments and column names. 
4. The challenges I had was mostly around if what I'm analyzing is useful for the purposes of my team. So i concluded to start with a simpler problem such as predict battery pack capacity which I hope to expand on when I have more time to gain more insightful information.

## Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challenging, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*
1. I liked that we were allowed to use our own dataset, made it more interesting for me and I know the topics we're better engrained that way for me. 
2. I also liked that we did the process from start to finish using pipelines, columntransformers and gridsearch since it is a lot more practical than running one model at a time and should be more reliable since we are doing the preprocessing after the data has been split each time. 
3. I find it very interesting to use what i learned in class on data im interested in and applicable to my extracurricular work, kept me motivated even though it extended the amount of work needed to do this assignment. 